## Install latest Obsei

In [1]:
!pip install git+https://github.com/lalitpagaria/obsei.git

  Cloning https://github.com/lalitpagaria/obsei.git to /tmp/pip-req-build-iec1ajft
  Running command git clone -q https://github.com/lalitpagaria/obsei.git /tmp/pip-req-build-iec1ajft
     |████████████████████████████████| 122kB 5.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 122kB 7.1MB/s 
     |████████████████████████████████| 184kB 7.5MB/s 
     |████████████████████████████████| 337kB 7.7MB/s 
     |████████████████████████████████| 81kB 5.0MB/s 
     |████████████████████████████████| 6.6MB 5.6MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 102kB 9.2MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 15.3MB 318kB/s 
     |████████████████████████████████| 163kB 41.3MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |██████████████████████████

## Configure Play Store Scrapper Source



In [2]:
from obsei.source.playstore_scrapper import PlayStoreScrapperConfig, PlayStoreScrapperSource

# initialize play store source config
source_config = PlayStoreScrapperConfig(
   # Need two parameters package_name and country. 
   # `package_name` can be found at the end of the url of app in play store. 
   # For example - https://play.google.com/store/apps/details?id=com.google.android.gm&hl=en&gl=US
   # `com.google.android.gm` is the package_name for xcode and `us` is country.
   countries=["us"],
   package_name="com.google.android.gm",
   max_count=10, # Number of reviews to fetch
   lookup_period="1h" # Lookup period from current time, format: `<number><d|h|m>` (day|hour|minute)

)

# initialize play store reviews retriever
source = PlayStoreScrapperSource()

## Configure Text Classification Analyzer

In [3]:
from obsei.analyzer.classification_analyzer import ClassificationAnalyzerConfig, ZeroShotClassificationAnalyzer

# initialize classification analyzer config
# It can also detect sentiments if "positive" and "negative" labels are added.
analyzer_config=ClassificationAnalyzerConfig(
   labels=["interface", "crash", "performance"],
)

# initialize classification analyzer
# For supported models refer https://huggingface.co/models?filter=zero-shot-classification
text_analyzer = ZeroShotClassificationAnalyzer(
   model_name_or_path="typeform/mobilebert-uncased-mnli",
   device="auto" # change to "cuda:0" for using gpu
)

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/98.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268 [00:00<?, ?B/s]

## Configure Logger Sink

In [5]:
from obsei.sink.logger_sink import LoggerSink, LoggerSinkConfig
import logging
import sys

logger = logging.getLogger("Obsei")
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# initialize logger sink config
sink_config = LoggerSinkConfig(
   logger=logger,
   level=logging.INFO
)

# initialize logger sink
sink = LoggerSink()

## Execute Workflow/Pipeline

In [6]:
# This will fetch information from configured source
source_response_list = source.lookup(source_config)

# This will execute analyzer
analyzer_response_list = text_analyzer.analyze_input(
    source_response_list=source_response_list,
    analyzer_config=analyzer_config
)

# This will send analyzed output to sink
sink.send_data(analyzer_response_list, sink_config)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


INFO:Obsei:{'processed_text': "Google all features are good but in google meet I'm facing one issue that I can't use any other app while google meet is going on. Pls check this and solve my problem. My email id avichauhan0101@gmail.com", 'segmented_data': {'negative': 0.4022720456123352, 'performance': 0.15470239520072937, 'interface': 0.09864232689142227, 'positive': 0.07919752597808838, 'crash': 0.012558789923787117}, 'meta': {'reviewId': 'gp:AOqpTOHwCPWVAoQa53rYp5AYLTgn6qvjTS9aMnoVmLDNmOED92zmuDZr2QeRhiCfzKK1CmM4WITHQM8IiAwyPs0', 'userName': 'Avi Chauhan', 'userImage': 'https://play-lh.googleusercontent.com/a-/AOh14Gj5iuXyHyxc3md1_CRApfikj5wk8r6-oDG_9G8Zaw', 'content': "Google all features are good but in google meet I'm facing one issue that I can't use any other app while google meet is going on. Pls check this and solve my problem. My email id avichauhan0101@gmail.com", 'score': 3, 'thumbsUpCount': 0, 'reviewCreatedVersion': '2021.03.21.366143478.Release', 'at': datetime.datetime